# 使用 LangChain 示例
sk-c64994ca9f9741888dc7bccdef2aa0a1

### 1. 获取 API key 并创建调用 API 的函数

首先使用`pip install dashscope`安装阿里云百炼的 Python SDK

然后导入相关依赖包，使用`getpass`输入api_key

In [77]:
from getpass import getpass
from dashscope import Generation
import os
# 安全获取API Key（输入时不显示明文）
api_key = getpass("请输入阿里云百炼API Key: ")
os.environ["DASHSCOPE_API_KEY"] = api_key
Generation.api_key = api_key


In [ ]:
def call_qwen(prompt="请介绍一下通义千问模型", model="qwen-turbo"):
    """
    调用通义千问模型获取回答

    Args:
        prompt: 用户输入的问题
        model: 使用的模型名称，默认为"qwen-turbo"

    Returns:
        模型生成的回答内容
    """
    try:
        # 调用模型生成文本（单轮对话用prompt参数）
        response = Generation.call(
            model="qwen-turbo",  # 指定模型
            prompt=prompt,  # 单轮对话使用prompt
            api_key=api_key  # 使用环境变量中的API Key
        )

        # 解析响应（output是字典，用["text"]获取内容）
        if response.status_code == 200:
            return response.output["text"]  # 修正解析方式
        else:
            return f"请求失败，状态码: {response.status_code}，原因: {response.message}"
    except Exception as e:
        return f"调用出错: {str(e)}"

# 测试调用
print(call_qwen("通义千问模型的特点是什么？"))

### 后端接口设计

- 导入本地文件
- 根据主题检索知识库文段
- 文段导入大模型生成QA问答
  - 单个文段导入大模型
  - 循环生成
- 处理生成的QA问答实现数据格式化
- 保存生成的题库至本地


In [ ]:
###导入本地文件函数
def import_local_file(file_path):
    """
    导入本地文件内容

    Args:
        file_path: 本地文件路径

    Returns:
        文件内容字符串
    """



### 2. 使用 LangChain 调用通义千问模型

In [80]:
from langchain_community.llms import Tongyi
# from langchain_community.llms import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [81]:
# 初始化模型
llm = Tongyi(
    model="qwen-turbo",
    temperature=0.7,
    max_tokens=1000,
    api_key=api_key
)

question = "什么是人工智能？"

# 创建提示模板
template = """
你是一个有用的助手。请回答以下问题：

问题：{question}

回答：
"""

prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

# 创建链
chain = LLMChain(llm=llm, prompt=prompt)

In [82]:
# 调用模型
question = "什么是人工智能？"
response = chain.invoke({"question": question})
print(response)

{'question': '什么是人工智能？', 'text': '人工智能（Artificial Intelligence，简称AI）是指由人创造的能够模拟人类智能行为的系统或机器。它可以通过学习、推理、感知、语言理解和问题解决等能力，完成一些通常需要人类智能才能完成的任务。\n\n人工智能可以分为两大类：\n\n1. **弱人工智能（Narrow AI）**：专注于执行特定任务，如语音识别、图像识别、推荐系统等。目前大多数实际应用都属于这一类，例如智能手机中的语音助手、搜索引擎和自动驾驶技术。\n\n2. **强人工智能（General AI）**：具备与人类相当的广泛认知能力，可以理解、学习和执行任何智力任务。目前尚未实现，仍处于研究阶段。\n\n人工智能的核心技术包括机器学习、深度学习、自然语言处理、计算机视觉和专家系统等。它在医疗、金融、交通、教育、娱乐等多个领域都有广泛应用。'}


### 3. 使用 faiss 构建RAG检索

使用`pip install pypdf faiss-cpu`安装依赖包

In [87]:
from langchain.document_loaders import PyPDFLoader

### 加载PDF文档
# loader = PyPDFLoader("./data/sample.pdf")
# pages = loader.load_and_split()

### 4. 将生成的题库保存到本地

In [84]:
import json
import uuid
from datetime import datetime

# 1. 定义保存路径
SAVE_DIR = "./question_banks"
import os
os.makedirs(SAVE_DIR, exist_ok=True)  # 确保文件夹存在

# 2. 生成单题数据
def create_question(stem, type_, answer, **kwargs):
    return {
        "question_id": str(uuid.uuid4()),  # 自动生成唯一ID
        "stem": stem,
        "type": type_,
        "answer": answer,
        "options": kwargs.get("options", []),
        "explanation": kwargs.get("explanation", ""),
        "difficulty": kwargs.get("difficulty", "medium"),
        "keywords": kwargs.get("keywords", []),
        "source": kwargs.get("source", ""),
        "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "updated_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

# 3. 保存题库到JSON
def save_bank_to_json(bank_name, questions, description=""):
    bank = {
        "bank_id": f"{bank_name.replace(' ', '_').lower()}_bank",
        "name": bank_name,
        "description": description,
        "questions": questions,
        "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "updated_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    file_path = os.path.join(SAVE_DIR, f"{bank_name}.json")
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(bank, f, ensure_ascii=False, indent=2)  # 缩进2空格，可读性强
    return file_path

# 4. 从JSON加载题库（用于刷题调用）
def load_bank_from_json(bank_name):
    file_path = os.path.join(SAVE_DIR, f"{bank_name}.json")
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)

# 示例：生成并保存一个题库
if __name__ == "__main__":
    # 创建2道题
    q1 = create_question(
        stem="线性回归的目标是最小化什么损失？",
        type_="single_choice",
        answer="A",
        options=["平方损失", "交叉熵损失", "绝对值损失"],
        difficulty="easy",
        keywords=["线性回归", "损失函数"],
        source="机器学习入门.pdf"
    )
    # 保存题库
    save_bank_to_json("机器学习基础题库", [q1])
    # 加载题库（刷题时用）
    bank = load_bank_from_json("机器学习基础题库")
    print("加载的题目：", bank["questions"][0]["stem"])

加载的题目： 线性回归的目标是最小化什么损失？


### 5. 使用 gradio 实现前端界面

In [ ]:
import gradio as gr



# 创建Gradio聊天界面
demo = gr.ChatInterface(
    fn=call_qwen,  # 传入函数对象，而非调用结果
    title="通义千问对话助手",
    description="与通义千问模型进行对话，支持多轮交流"
    
)

# 启动界面
if __name__ == "__main__":
    demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7885

Could not create share link. Missing file: d:\anaconda\envs\chatgpt\lib\site-packages\gradio\frpc_windows_amd64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_windows_amd64.exe
2. Rename the downloaded file to: frpc_windows_amd64_v0.2
3. Move the file to this location: d:\anaconda\envs\chatgpt\lib\site-packages\gradio
